In [8]:
# Create matrix
import numpy as np
cost = np.random.rand(48,91,91) * 15

In [20]:
# Heuristic nearest neighbor

import numpy as np
import math

#cost = np.random.rand(48,91,91) * 15

cities = [i for i in range(40)]
#cities = [1,5,12,15,16,22,35,46]


compare = 100
timeslot = 0
path = [0] * (len(cities) + 1)
print(path)
depth = 0
time = 0


# Calculate first edge
for i in cities:
    for j in cities:
        if i != j:
            if cost[timeslot][i][j] < compare:
                compare = cost[timeslot][i][j]
                path[depth] = i
                path[len(path) -1 ] = i
                path[depth + 1] = j
depth = 2
time += cost[timeslot][path[0]][path[1]]
print(path, time)

# !!! change that they take the timeslot if time exceeds the window



for i in cities:
    if i != path[0] and i != path[1]:
        timeslot = math.floor(time / 15)
        compare = 100
        for j in cities:
            if (compare > cost[timeslot][path[depth]][j] and path[depth] != j) and j not in path[0:depth]:
                compare = cost[timeslot][depth][j]
                path[depth] = j

        time += cost[timeslot][path[depth]][path[depth]]
        depth += 1
        print(path, timeslot, time)

time += cost[timeslot][path[len(path) -1]][path[0]]
print('finished', path, time)
        
    

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17] 0.0007908446710203076
[17, 3, 37, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17] 0 0.5390423123879257
[17, 3, 37, 35, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17] 0 13.187574370719652
[17, 3, 37, 35, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17] 0 13.711632971532191
[17, 3, 37, 35, 38, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17] 0 28.480121200284074
[17, 3, 37, 35, 38, 36, 39, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17]

In [17]:
# Solving time dependent CPLEX

import numpy as np
from docplex.mp.model import Model
import docplex.mp.solution as Solution


#cost = np.loadtxt('dependent_2017to2019_15min_withouthashtag.txt')
#cost = np.reshape(cost,(48,91,91))
#cost = np.random.rand(48,91,91) * 10

#print(cost)

cities= [1,5,12,15,16,22,35,46]
cities = [ i for i in range(40)]

#print(cities)
arcs =[(t,i,j) for t in range(len(cost)) for i in cities for j in cities if i!=j]


distance={(t,i,j): cost[t,i,j] for t,i,j in arcs}
#print(distance[(0,1,2)])

# CPLEX model
mdl=Model('TSP')

# decision variables

x=mdl.binary_var_dict(arcs,name='x')
d=mdl.continuous_var_dict(cities,name='d')



mdl.minimize(mdl.sum(distance[i]*x[i] for i in arcs))

# Constraints

for c in cities:
    mdl.add_constraint(mdl.sum(x[(t,i,j)] for t,i,j in arcs if i==c)==1, ctname='out_%d'%c)
    
for c in cities:
    mdl.add_constraint(mdl.sum(x[(t,i,j)] for t,i,j in arcs if j==c)==1, ctname='in_%d'%c)
    
for timeslot in range(len(cities)):
    mdl.add_constraint(mdl.sum(x[(t,i,j)] for t,i,j in arcs if t==timeslot)==1, ctname='time_%d'%c)
    
# Logical time-following    
for t,i,j in arcs:
    if j!=0 and t < len(cities) -1:
        mdl.add_indicator(x[(t,i,j)], (mdl.sum(x[(t + 1,j,k)]for k in cities if k !=j) == 1), name='new_(%d,_%d)'%(i, j)) 
        
# starting point

#mdl.add_constraint(mdl.sum(x[(0,cities[0],j)] for j in cities if j !=cities[0]) == 1, ctname = 'starting constraint')

    
print(mdl.export_to_string())

mdl.parameters.timelimit=120
mdl.parameters.mip.strategy.branch=1
mdl.parameters.mip.tolerances.mipgap=0.15

solution = mdl.solve(log_output=True)

mdl.get_solve_status()

solution.display()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
CPXPARAM_MIP_Strategy_Branch                     1
CPXPARAM_TimeLimit                               120
CPXPARAM_MIP_Tolerances_MIPGap                   0.14999999999999999
Tried aggregator 2 times.
MIP Presolve eliminated 0 rows and 10960 columns.
MIP Presolve modified 57798 coefficients.
Aggregator did 57798 substitutions.
Reduced MIP has 60960 rows, 65481 columns, and 369798 nonzeros.
Reduced MIP has 63960 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.47 sec. (7823.46 ticks)
Probing time = 0.34 sec. (30.34 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 57798 rows and 0 columns.
MIP Presolve modified 1521 coefficients.
Reduced MIP has 3162 rows, 65481 columns, and 312000 nonzeros.
Reduced MIP has 65481 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.41 sec. (789.57 ticks)
Probing time = 0.23 sec. (34.87 ti

In [66]:
#######################################


In [16]:
# Solving time independent

import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.model import Model
import docplex.mp.solution as Solution

# for sequentiel solving 

from itertools import product

#cost = np.loadtxt('matrix_time-independent_2017to2019_delete.txt')
cost = np.random.rand(91,91) * 10

#cities=[i for i in range(len(cost))]

cities = [7,15,22,35,53]
#print(cities)
arcs =[(i,j) for i in cities for j in cities if i!=j]


distance={(i, j): cost[i,j] for i,j in arcs}
#print(distance)

# CPLEX model
mdl=Model('TSP')

# decision variables

x=mdl.binary_var_dict(arcs,name='x')
d=mdl.continuous_var_dict(cities,name='d')

mdl.minimize(mdl.sum(distance[i]*x[i] for i in arcs))

# Constraints
for c in cities:
    mdl.add_constraint(mdl.sum(x[(i,j)] for i,j in arcs if i==c)==1, 
                       ctname='out_%d'%c)
    
for c in cities:
    mdl.add_constraint(mdl.sum(x[(i,j)] for i,j in arcs if j==c)==1, 
                       ctname='in_%d'%c)

    
# conventional way to eliminate subtours (Dantzig, Fulkerson and Johnson) !!!DOES NOT WORK!!!
#for i,j in arcs:
    #if j!=cities[0]:
        #mdl.add_indicator(x[(i,j)],(mdl.sum(x[(j,k)] for k in cities if j !=k) ==1 , name='order_(%d,_%d)'%(i, j))
        

#sequential way to eliminate subtours (Miller, Tucker and Zemlin)
for i,j in arcs:
    if j!=cities[0]:
        mdl.add_indicator(x[(i,j)],d[i]+1==d[j], name='order_(%d,_%d)'%(i, j))
        
        
print(mdl.export_to_string())

# possible to add constraint

mdl.parameters.timelimit=120
mdl.parameters.mip.strategy.branch=1
mdl.parameters.mip.tolerances.mipgap=0.15

solution = mdl.solve(log_output=True)

mdl.get_solve_status()

solution.display()


SyntaxError: invalid syntax (<ipython-input-16-dcc4451f8e95>, line 48)

In [16]:
# testdata

import numpy as np

test = np.random.rand(10,10) * 100

print(test)

np.savetxt('test.txt', test, fmt='%-4.2f')

[[29.4347901   8.20758003 29.10196074 72.26923591 22.992751   12.49573891
  19.73883223 22.89879464 11.79526477 72.54448897]
 [85.95343714 42.86686349  7.63407884 40.29326144  2.04140725 78.27410305
   7.96230243 19.9709607  47.97561912  0.1324633 ]
 [64.36273156 13.67748046 51.13256728 67.96376069 49.24307182 69.9642299
  14.77743171 89.17367447 99.1918     88.03598032]
 [84.08563954  5.76651227 28.02890045 50.57124431 64.65656705 29.82816721
  70.84065227 78.28381715 90.31745547 91.4240113 ]
 [92.71868447 34.89122347  3.92211883  9.00333898 28.04198931 73.19203986
  76.89485575 89.08057956 40.88895342 88.72292306]
 [15.33648129 80.13912192 86.99239293 11.47276369 26.73041584 88.53223331
  56.46494295 42.39599126 56.68660673  5.2528669 ]
 [ 0.62580905 30.48535235 23.39955139 78.6771767  81.88049763 81.52303494
  11.78178821 44.80112531 76.33897545 10.5030909 ]
 [40.63860027 81.32164563 65.66464617 83.3495674  13.21381383 70.01372946
  69.32495192 55.01488108 67.52053148 34.84805028]
 

In [ ]:
Implement sequential formulation in TSP (Miller, Tucker, Zemlin)